In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train_data_task1.csv
/kaggle/input/dev_data_task3.csv
/kaggle/input/train_data_task2.csv
/kaggle/input/dev_data_task1.csv
/kaggle/input/train_data_task3.csv
/kaggle/input/EXIT2024_tweet_test.csv
/kaggle/input/dev_data_task2.csv


In [41]:
!rm -rf /kaggle/working/output/best-model/*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
!wandb offline
!wandb disabled

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
W&B disabled.


In [3]:
!pip install datasets
!pip install transformers

In [4]:
from  IPython. display import clear_output
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm

import torch
import torch.optim as optim
import torch. nn.functional as F
import re
import warnings
warnings.filterwarnings('ignore')

2024-05-02 07:14:35.214431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 07:14:35.214524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 07:14:35.313782: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
train_df = pd.read_csv('/kaggle/input/train_data_task1.csv')
test_df = pd.read_csv('/kaggle/input/EXIT2024_tweet_test.csv')
dev_df = pd.read_csv('/kaggle/input/dev_data_task1.csv')

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2076 entries, 0 to 2075
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2076 non-null   int64 
 1   lang    2076 non-null   object
 2   tweet   2076 non-null   object
 3   split   2076 non-null   object
dtypes: int64(1), object(3)
memory usage: 65.0+ KB


In [9]:
test_df.head()

,id,lang,tweet,split
0,500001,es,@Eurogamer_es Todo gamergate desde el desarrol...,TEST_ES
1,500002,es,"@ArCaNgEl__23 @Benzenazi Hombre, no es compara...",TEST_ES
2,500003,es,yo buscando las empresas metidas en el gamerga...,TEST_ES
3,500004,es,"@jordirico Primero fue internet, luego el game...",TEST_ES
4,500005,es,@AlonsoQuijano12 Yo estuve metido en el gamerg...,TEST_ES


In [6]:
def simple_preprocess(text):
    """
    pass the tweet data as a series. do not use apply function
    only preprocesses for replacing @USER and URLS
    """
    URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
    HANDLE_RE = re.compile(r"@\w+")
    tweets = []
    for t in text:
        t = HANDLE_RE.sub("@user", t)
        t = URL_RE.sub("http", t)
        tweets.append(t)
    return tweets

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os.path
from os import path
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

# Hard

In [8]:
#instantiate label encoders
task1_encoder = LabelEncoder()

def task1_hard_encode(df):
    task1_encoder.fit(all_task1_hard_labels)
    df['hard_label'] = task1_encoder.transform(df['hard_label'])
    return df

def task1_hard_decode(df):
    df["hard_label"] = task1_encoder.inverse_transform(df["hard_label"])
    return df

In [9]:
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

all_task1_hard_labels = pd.concat([og_train1["hard_label"], og_dev1["hard_label"]])
train1_df = task1_hard_encode(og_train1)

# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 2]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

dev1_df = task1_hard_encode(og_dev1)
dev1_df = dev1_df[["tweet","hard_label"]].dropna()
dev1_df = dev1_df[dev1_df['hard_label'] != 2]
dev1_df["tweet"] = simple_preprocess(dev1_df["tweet"])

test1_df = og_test1
test1_df = test1_df[["tweet"]]
# test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (6064, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @user Ignora al otro, es un capullo.El problem...           1
1     @user Si comicsgate se parece en algo a gamerg...           0
2     @user Lee sobre Gamergate, y como eso ha cambi...           0
4     @user @user @user Entonces como así es el merc...           1
5     @user Aaah sí. Andrew Dobson. El que se dedicó...           0
...                                                 ...         ...
6915  idk why y’all bitches think having half your a...           1
6916  This has been a part of an experiment with @us...           1
6917  "Take me already" "Not yet. You gotta be ready...           1
6918       @user why do you look like a whore? /lh http           1
6919  ik when mandy says “you look like a whore” i l...           1

[6064 rows x 2 columns]>
test1 (2076, 1)
<bound method NDFrame.head of                                                   tweet
0     @us

In [10]:
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# with GPT2

In [11]:
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


In [12]:
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)
# tokenizer = AutoTokenizer.from_pretrained('openai-community/openai-gpt')
# model = AutoModelForSequenceClassification.from_pretrained("openai-community/openai-gpt", num_labels=2,ignore_mismatched_sizes=True)

print('Loading gpt-2 model')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path='gpt2', num_labels=2)

print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2')
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path='gpt2', config=model_config)
model.resize_token_embeddings(len(tokenizer)) 
model.config.pad_token_id = model.config.eos_token_id
model.to(device)

Loading gpt-2 model


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loading model...


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [24]:
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)
# tokenizer = AutoTokenizer.from_pretrained('openai-community/openai-gpt')
# model = AutoModelForSequenceClassification.from_pretrained("openai-community/openai-gpt", num_labels=2,ignore_mismatched_sizes=True)

print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path='ai-forever/mGPT')
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

print('Loading model...')
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path='ai-forever/mGPT', num_labels=2)
model.resize_token_embeddings(len(tokenizer)) 
model.config.pad_token_id = model.config.eos_token_id
model.to(device)

Loading tokenizer...
Loading model...


pytorch_model.bin:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ai-forever/mGPT and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(100000, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=2048, out_features=2, bias=False)
)

In [13]:
from datasets import load_dataset, DatasetDict


# Training arguments for GPT large

https://datascience.stackexchange.com/questions/64583/what-are-the-good-parameter-ranges-for-bert-hyperparameters-while-finetuning-it

In [25]:
# Set parameters
MAX_LENGTH = 256

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model-mgpt",
    report_to=None,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [22]:
# Set parameters
MAX_LENGTH = 256

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model",
    report_to=None,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [46]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model",
    report_to=None,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [53]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model",
    report_to=None,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=4e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [15]:
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

In [26]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)

# Create the datasets
train_encodings = tokenizer(train_dataset["text"],return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], padding=True, truncation=True, max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], padding=True, truncation=True, max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})

In [17]:
def compute_f1_score(pred):
    # pred is a tuple (predictions, labels)
    predictions, labels = pred
    # Compute the F1 score
    f1 = f1_score(labels, predictions.argmax(axis=1), average='macro')
    return {"f1_score": f1}

optimizers (Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR], optional, defaults to (None, None)) — A tuple containing the optimizer and the scheduler to use. Will default to an instance of AdamW on your model and a scheduler given by get_linear_schedule_with_warmup() controlled by args.

# with lr=1e-5

In [18]:
# model.to(device)
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss,F1 Score
500,0.701400,0.653078,0.626795
1000,0.642900,0.556140,0.709708
1500,0.539400,0.544216,0.728638


TrainOutput(global_step=1860, training_loss=0.5939103403399068, metrics={'train_runtime': 645.6891, 'train_samples_per_second': 46.059, 'train_steps_per_second': 2.881, 'total_flos': 1942741340651520.0, 'train_loss': 0.5939103403399068, 'epoch': 5.0})

In [27]:
# mGPT
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score
)

# Train the model for each fold
trainer.train()

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/models/gpt2/modeling_gpt2.py", line 1426, in forward
    transformer_outputs = self.transformer(
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **

# with lr=3e-5

In [49]:
# model.to(device)
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss,F1 Score
500,0.402800,0.275215,0.900871
1000,0.227700,0.385791,0.904570
1500,0.091400,0.404047,0.913214


TrainOutput(global_step=1860, training_loss=0.18115151825771536, metrics={'train_runtime': 710.327, 'train_samples_per_second': 41.868, 'train_steps_per_second': 2.619, 'total_flos': 1956230696601600.0, 'train_loss': 0.18115151825771536, 'epoch': 5.0})

# with lr=4e-5

In [54]:
# model.to(device)
# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_f1_score
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss,F1 Score
500,0.248800,0.545815,0.859971
1000,0.206200,0.502150,0.877194
1500,0.066700,0.529500,0.899541


TrainOutput(global_step=1860, training_loss=0.13659687965146958, metrics={'train_runtime': 711.1077, 'train_samples_per_second': 41.822, 'train_steps_per_second': 2.616, 'total_flos': 1956230696601600.0, 'train_loss': 0.13659687965146958, 'epoch': 5.0})

# with lr=1e-5

In [19]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score

# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"]

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels)

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.7295238095238096
Validation F1 Score: 0.7441441441441442


# with lr=3e-05

In [50]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score

# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"]

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels)

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.900952380952381
Validation F1 Score: 0.8980392156862745


# with lr=4e-5

In [55]:
from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score

# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"]

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels)

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.8790476190476191
Validation F1 Score: 0.8621064060803475


In [20]:
# Get predictions on the test set
from scipy.special import softmax
test_predictions = trainer.predict(test_dataset)
test_pred_labels = np.argmax(test_predictions.predictions, axis=1)
test_pred_probs = softmax(test_predictions.predictions).tolist()

#get probabilities to a list
test_probs_list = [] 
for logits in test_pred_probs:
    test_probs_list.append({'YES': logits[0], 'NO': logits[1]})

In [21]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task1_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "./output/task1_NICA-hard-8.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels,columns=["hard_label"]), test_probs_list, og_test1[["id"]],og_test1)

decoded labels length: 2076
output generated as json


# Soft

In [41]:
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"computation will run on {device} now")

computation will run on cuda:0 now


In [28]:
#instantiate label encoders
def convert_logits_to_list(logits_dict):
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["YES"], logits_dict["NO"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"YES": logits_list[0], "NO": logits_list[1]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

In [59]:
train_df = pd.read_csv('/kaggle/input/exist-2024-clef/train_data_task1.csv')
test_df = pd.read_csv('/kaggle/input/exist-2024-clef/EXIT2024_tweet_test.csv')
dev_df = pd.read_csv('/kaggle/input/exist-2024-clef/dev_data_task1.csv')

In [60]:
og_train1 = train_df.copy()
og_dev1 = dev_df.copy()
og_test1 = test_df.copy()

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
train1_df = og_train1[["tweet","soft_label"]].dropna()


og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
dev1_df = og_dev1[["tweet","soft_label"]].dropna()

test1_df = og_test1
test1_df = test1_df[["tweet"]]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

train1 (6920, 2)
<bound method NDFrame.head of                                                   tweet  \
0     @user Ignora al otro, es un capullo.El problem...   
1     @user Si comicsgate se parece en algo a gamerg...   
2     @user Lee sobre Gamergate, y como eso ha cambi...   
3     @user Un retraso social bastante lamentable, g...   
4     @user @user @user Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @us...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918       @user why do you look like a whore? /lh http   
6919  ik when mandy says “you look like a whore” i l...   

                                     soft_label  
0      [0.833333333333333, 0.16666666666666602]  
1      [0.16666666666666602, 0.833333333333333]  
2                                    [0.0, 1.0]  
3                                    [0.5, 0

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [32]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=2,ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# with lr=2e-5

In [33]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model/soft/",
    report_to=None,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

# with lr=3e-5

In [63]:
# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output/best-model/soft/",
    report_to=None,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_total_limit = 1,
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

In [64]:
import torch.nn as nn

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

In [65]:
# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})

In [36]:
def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss

# with lr=2e-5

In [37]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


# Define the trainer for each fold
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss
500,10.089100,19.161743
1000,9.503400,19.068266
1500,9.200000,19.000187
2000,9.014700,18.973082


TrainOutput(global_step=2115, training_loss=9.341863595227542, metrics={'train_runtime': 816.0835, 'train_samples_per_second': 41.442, 'train_steps_per_second': 2.592, 'total_flos': 2224603973068800.0, 'train_loss': 9.341863595227542, 'epoch': 5.0})

# with lr=3e-5

In [66]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


# Define the trainer for each fold
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

Step,Training Loss,Validation Loss
500,9.301600,18.027428
1000,8.903200,18.022764
1500,8.802900,17.967220
2000,8.740700,17.960915


TrainOutput(global_step=2115, training_loss=8.887947071697695, metrics={'train_runtime': 816.7203, 'train_samples_per_second': 41.41, 'train_steps_per_second': 2.59, 'total_flos': 2224603973068800.0, 'train_loss': 8.887947071697695, 'epoch': 5.0})

# with lr=2e-5

In [38]:

from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_logit_labels = val_predictions.predictions
val_pred_probabilities = softmax(val_pred_logit_labels, axis=1)

val_pred_hard_labels = np.argmax(val_pred_probabilities, axis=1)
val_true_labels = val_dataset["label"]
val_true_hard_labels = np.argmax(val_dataset["label"],axis=1)

# Calculate evaluation metrics for each fold
val_accuracy = accuracy_score(val_true_hard_labels, val_pred_hard_labels)
val_f1_score = f1_score(val_true_hard_labels, val_pred_hard_labels,average='weighted')

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.821608040201005
Validation F1 Score: 0.821529522778418


# with lr=3e-5

In [67]:

from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_logit_labels = val_predictions.predictions
val_pred_probabilities = softmax(val_pred_logit_labels, axis=1)

val_pred_hard_labels = np.argmax(val_pred_probabilities, axis=1)
val_true_labels = val_dataset["label"]
val_true_hard_labels = np.argmax(val_dataset["label"],axis=1)

# Calculate evaluation metrics for each fold
val_accuracy = accuracy_score(val_true_hard_labels, val_pred_hard_labels)
val_f1_score = f1_score(val_true_hard_labels, val_pred_hard_labels,average='weighted')

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

Validation Accuracy: 0.9128978224455612
Validation F1 Score: 0.9126874946611928


In [68]:
# Get predictions on the test set
from scipy.special import softmax
test_predictions1 = trainer.predict(test_dataset)
test_pred_labels1 = np.argmax(test_predictions1.predictions, axis=1)
test_pred_probs1 = softmax(test_predictions1.predictions).tolist()

#get probabilities to a list
test_probs_list1 = [] 
for logits in test_pred_probs1:
    test_probs_list1.append({'YES': logits[0], 'NO': logits[1]})

In [69]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id"].astype(int)
    output_dict = {}
    decoded_labels = task1_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "task1_NICA-soft-3.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels,columns=["hard_label"]), test_probs_list1, og_test1[["id"]],og_test1)

decoded labels length: 2076
output generated as json
